In [34]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [35]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [36]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


On charge le dataset et on effectue une première briève analyse sur le dataset

In [37]:
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
dataset = load_dataset("julien-c/titanic-survival")
df = pd.DataFrame(dataset['train'])


In [38]:
print(df.shape)

(887, 8)


In [39]:
#On regarde si on a des données manquantes ou des données dupliquées

print(df.isnull().sum())
print(df.duplicated().sum())

Survived                   0
Pclass                     0
Name                       0
Sex                        0
Age                        0
Siblings/Spouses Aboard    0
Parents/Children Aboard    0
Fare                       0
dtype: int64
0


Création de nouvelles features

In [40]:
pip install altair

Note: you may need to restart the kernel to use updated packages.


In [41]:
import altair as alt

on va ensuite chercheer à dériver de nouvelles features des features existantes en s'aidant de visualisations

In [42]:
df['FamilySize'] = df['Siblings/Spouses Aboard'] + df['Parents/Children Aboard'] + 1

# Création du graphique
chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('FamilySize:O', title='Taille de la famille'),
    y=alt.Y('count()', title='Nombre de passagers'),
    color=alt.Color('Survived:N', title='Survécu', scale=alt.Scale(domain=[0, 1], range=['red', 'green'])),
    tooltip=[alt.Tooltip('count()', title='Nombre de passagers'), 'FamilySize', 'Survived']
).properties(
    title='Survie en fonction de la taille de la famille sur le Titanic'
)

chart.display()



alt.Chart(...)

In [43]:
def categorize_family_size(size):
    if size == 1:
        return 'Solo'
    elif size <= 4:
        return 'Petite Famille'
    else:
        return 'Grande Famille'

df['FamilyType'] = df['FamilySize'].apply(categorize_family_size)

# Création du graphique
chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('FamilyType:O', title='Taille de la famille'),
    y=alt.Y('count()', title='Nombre de passagers'),
    color=alt.Color('Survived:N', title='Survécu', scale=alt.Scale(domain=[0, 1], range=['red', 'green'])),
    tooltip=[alt.Tooltip('count()', title='Nombre de passagers'), 'FamilyType', 'Survived']
).properties(
    title='Survie en fonction de la taille de la famille sur le Titanic'
)

chart.display()

alt.Chart(...)

In [44]:
#encodage par etiquettes pour FamilyType

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['FamilyType'] = label_encoder.fit_transform(df['FamilyType'])

In [45]:
import re

def extract_title(name):
    title_search = re.search('([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return " "
df['Title'] = df['Name'].apply(extract_title)

# Création du graphique
chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Title:O', title='Titre'),
    y=alt.Y('count()', title='Nombre de passagers'),
    color=alt.Color('Survived:N', title='Survécu', scale=alt.Scale(domain=[0, 1], range=['red', 'green'])),
    tooltip=[alt.Tooltip('count()', title='Nombre de passagers'), 'Title', 'Survived']
).properties(
    title='Survie en fonction du titre sur le Titanic'
)

chart.display()

<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
/var/folders/90/vkxt4j7s77jgmpj5lqchw0bh0000gn/T/ipykernel_16116/1711318172.py:4: SyntaxWarning: invalid escape sequence '\.'
  title_search = re.search('([A-Za-z]+)\.', name)


alt.Chart(...)

In [46]:
def group_titles(title):
    if title in ['Capt', 'Col', 'Countess', 'Don', 'Jonkheer', 'Lady', 'Sir']:
        return 'Royalty'
    elif title in ['Major']:
        return 'Military'
    elif title in ['Dr', 'Rev']:
        return 'Professional'
    elif title in ['Miss', 'Mlle', 'Ms']:
        return 'Miss'
    elif title in ['Mrs', 'Mme']:
        return 'Mrs'
    elif title == 'Master':
        return 'Master'
    elif title == 'Mr':
        return 'Mr'
    else:
        return title  # Pour les cas non listés

# Applique la fonction de regroupement à la colonne des titres
df['Title'] = df['Title'].apply(group_titles)

In [47]:
print(df['Title'].value_counts())

# Création du graphique
chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Title:O', title='Titre'),
    y=alt.Y('count()', title='Nombre de passagers'),
    color=alt.Color('Survived:N', title='Survécu', scale=alt.Scale(domain=[0, 1], range=['red', 'green'])),
    tooltip=[alt.Tooltip('count()', title='Nombre de passagers'), 'Title', 'Survived']
).properties(
    title='Survie en fonction du titre sur le Titanic'
)

chart.display()

Title
Mr              513
Miss            185
Mrs             126
Master           40
Professional     13
Royalty           8
Military          2
Name: count, dtype: int64


alt.Chart(...)

In [48]:
#name length

df['NameLength'] = df['Name'].apply(len)

# Création du graphique
chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('NameLength:O', title='Longueur du nom'),
    y=alt.Y('count()', title='Nombre de passagers'),
    color=alt.Color('Survived:N', title='Survécu', scale=alt.Scale(domain=[0, 1], range=['red', 'green'])),
    tooltip=[alt.Tooltip('count()', title='Nombre de passagers'), 'NameLength', 'Survived']
).properties(
    title='Survie en fonction de la longueur du nom sur le Titanic'
)

chart.display()

alt.Chart(...)

In [49]:
def categorize_age(age):
    if age <= 12:
        return 'Child'
    elif age <= 60:
        return 'Adult'
    else:
        return 'Senior'

df['AgeGroup'] = df['Age'].apply(categorize_age)

# Création du graphique
chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('AgeGroup:O', title='Groupe d\'âge'),
    y=alt.Y('count()', title='Nombre de passagers'),
    color=alt.Color('Survived:N', title='Survécu', scale=alt.Scale(domain=[0, 1], range=['red', 'green'])),
    tooltip=[alt.Tooltip('count()', title='Nombre de passagers'), 'AgeGroup', 'Survived']
).properties(
    title='Survie en fonction du groupe d\'âge sur le Titanic'
)

chart.display()

alt.Chart(...)

In [50]:
#encodage par etiquettes pour AgeGroup

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['AgeGroup'] = label_encoder.fit_transform(df['AgeGroup'])




In [51]:
df['Sex'] = df['Sex'].apply(lambda x: 1 if x == 'female' else 0)



In [52]:
#onehot encoding de Title

df = pd.get_dummies(df, columns=['Title'])



In [53]:
df['TicketFrequency'] = df['Fare'].map(df['Fare'].value_counts())


On créé ensuite la feature FamilySurvivalRate qui correspont simplement pour chaque individu au taux de survie observé dans l'ensemble d'entrainement pour sa famille (sans prendre l'individu même dans le calcul)

On rajoute également un coefficient  de ConfidenceFamilySurvival qui correspond au nombre de personne sur lequel on base ce calcul.

In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Préparation des données
df['Family'] = df['Name'].str.extract('(\w+)(?:,|$)', expand=False)
df_train, df_test = train_test_split(df, test_size=0.2, random_state=0)

# Reset des index pour une manipulation correcte des données
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

# Fonction pour pré-calculer les taux de survie familiaux sans inclure l'individu
def calculate_family_survival_stats(data):
    family_stats = {}
    for idx, row in data.iterrows():
        key = row['Family']
        if key not in family_stats:
            family_stats[key] = {'total_survived': 0, 'count': 0}
        family_stats[key]['total_survived'] += row['Survived']
        family_stats[key]['count'] += 1

    # Convertir en DataFrame
    family_survival_stats = pd.DataFrame([
        {'Family': k, 'TotalSurvived': v['total_survived'], 'FamilyCount': v['count']}
        for k, v in family_stats.items()
    ])
    return family_survival_stats

# Appliquer la fonction pour obtenir les statistiques de survie de la famille sur l'ensemble d'entraînement
family_survival_stats = calculate_family_survival_stats(df_train)

# Fonction pour appliquer les statistiques de survie de la famille aux données
def apply_family_survival(data, train_data):
    result_rows = []
    for idx, row in data.iterrows():
        family_members = train_data[train_data['Family'] == row['Family']]
        adjusted_family_members = family_members[family_members.index != idx]

        adjusted_survived = adjusted_family_members['Survived'].sum()
        adjusted_count = len(adjusted_family_members)

        family_survival_rate = adjusted_survived / adjusted_count if adjusted_count > 0 else 0.5
        confidence_family_survival = adjusted_count

        result_row = row.to_dict()
        result_row['FamilySurvivalRate'] = family_survival_rate
        result_row['ConfidenceFamilySurvival'] = confidence_family_survival
        result_rows.append(result_row)

    return pd.DataFrame(result_rows)

# Appliquer les statistiques de survie aux ensembles de données
df_train = apply_family_survival(df_train, df_train)
df_test = apply_family_survival(df_test, df_train)

# Ajouter une colonne pour indiquer l'origine des données
df_train['Source'] = 'train'
df_test['Source'] = 'test'

# Combiner les deux DataFrames
df_combined = pd.concat([df_train, df_test])

# Choisir aléatoirement 100 familles uniques
import random
sampled_families = random.sample(list(df_combined['Family'].unique()), 100)

# Afficher les données pour ces familles
for family in sampled_families:
    family_data = df_combined[df_combined['Family'] == family]
    display(family_data[[
        'Name', 'Family', 'Survived', 'FamilySize', 'FamilySurvivalRate', 'ConfidenceFamilySurvival', 'Source'
    ]])

# Vérifier si les colonnes des taux de survie et autres sont correctement formées
print("Colonnes présentes dans df_combined:", df_combined.columns)




<>:5: SyntaxWarning: invalid escape sequence '\w'
<>:5: SyntaxWarning: invalid escape sequence '\w'
/var/folders/90/vkxt4j7s77jgmpj5lqchw0bh0000gn/T/ipykernel_16116/3031268366.py:5: SyntaxWarning: invalid escape sequence '\w'
  df['Family'] = df['Name'].str.extract('(\w+)(?:,|$)', expand=False)


,Name,Family,Survived,FamilySize,FamilySurvivalRate,ConfidenceFamilySurvival,Source
672,Mr. Ingvald Olai Olsen Hagland,Hagland,0,2,0.5,0,train
148,Mr. Konrad Mathias Reiersen Hagland,Hagland,0,2,0.0,1,test


,Name,Family,Survived,FamilySize,FamilySurvivalRate,ConfidenceFamilySurvival,Source
289,Mr. Emile Richard,Richard,0,1,0.5,0,train


,Name,Family,Survived,FamilySize,FamilySurvivalRate,ConfidenceFamilySurvival,Source
3,Mr. Juho Stranden,Stranden,1,1,0.5,0,test


,Name,Family,Survived,FamilySize,FamilySurvivalRate,ConfidenceFamilySurvival,Source
11,Miss. Marjorie Newell,Newell,1,2,0.5,0,train
82,Mr. Arthur Webster Newell,Newell,0,3,1.0,1,test
109,Miss. Madeleine Newell,Newell,1,2,1.0,1,test


...

,Name,Family,Survived,FamilySize,FamilySurvivalRate,ConfidenceFamilySurvival,Source
67,Mrs. William (Imanita Parrish Hall) Shelley,Shelley,1,2,0.5,0,test


,Name,Family,Survived,FamilySize,FamilySurvivalRate,ConfidenceFamilySurvival,Source
506,Mr. Fletcher Fellows Williams-Lambert,Lambert,0,1,0.5,0,train


,Name,Family,Survived,FamilySize,FamilySurvivalRate,ConfidenceFamilySurvival,Source
591,Mr. William Thomas Beavan,Beavan,0,1,0.5,0,train


,Name,Family,Survived,FamilySize,FamilySurvivalRate,ConfidenceFamilySurvival,Source
55,Mr. Thomas Drake Martinez Cardeza,Cardeza,1,2,0.5,0,train


Colonnes présentes dans df_combined: Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings/Spouses Aboard',
       'Parents/Children Aboard', 'Fare', 'FamilySize', 'FamilyType',
       'NameLength', 'AgeGroup', 'Title_Master', 'Title_Military',
       'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Professional',
       'Title_Royalty', 'TicketFrequency', 'Family', 'FamilySurvivalRate',
       'ConfidenceFamilySurvival', 'Source'],
      dtype='object')


In [55]:
import random
import pandas as pd

# Ajouter une colonne pour indiquer l'origine des données
df_train['Source'] = 'train'
df_test['Source'] = 'test'

# Combiner les deux DataFrames
df = pd.concat([df_train, df_test])

# Choisir aléatoirement 100 familles uniques
sampled_families = random.sample(list(df['Family'].unique()), 100)

# Afficher les données pour ces familles
for family in sampled_families:
    family_data = df[df['Family'] == family]
    display(family_data[[
        'Name', 'Family', 'Survived', 'FamilySize', 'FamilySurvivalRate', 'ConfidenceFamilySurvival', 'Source',
    ]])

# Vérifier si les colonnes des taux de survie et autres sont correctement formées
print("Colonnes présentes dans df:", df.columns)


,Name,Family,Survived,FamilySize,FamilySurvivalRate,ConfidenceFamilySurvival,Source
174,Mr. Gustaf Hjalmar Edvardsson,Edvardsson,0,1,0.5,0,test


,Name,Family,Survived,FamilySize,FamilySurvivalRate,ConfidenceFamilySurvival,Source
104,Master. Viljo Hamalainen,Hamalainen,1,3,1.0,1,train
267,Mrs. William (Anna) Hamalainen,Hamalainen,1,3,1.0,1,train


,Name,Family,Survived,FamilySize,FamilySurvivalRate,ConfidenceFamilySurvival,Source
62,Miss. Marion Louise Becker,Becker,1,4,1.0,1,train
564,Master. Richard F Becker,Becker,1,4,1.0,1,train


,Name,Family,Survived,FamilySize,FamilySurvivalRate,ConfidenceFamilySurvival,Source
494,Mr. Hans Kristensen Givard,Givard,0,1,0.5,0,train


,Name,Family,Survived,FamilySize,FamilySurvivalRate,ConfidenceFamilySurvival,Source
413,Mrs. Joseph (Maria Elias) Caram,Caram,0,2,0.5,0,train


...

,Name,Family,Survived,FamilySize,FamilySurvivalRate,ConfidenceFamilySurvival,Source
141,Mr. Mitto Denkoff,Denkoff,0,1,0.5,0,train


,Name,Family,Survived,FamilySize,FamilySurvivalRate,ConfidenceFamilySurvival,Source
423,Miss. Doolina Margaret Ford,Ford,0,5,0.0,1,train
441,Miss. Robina Maggie Ford,Ford,0,5,0.0,1,train
112,Mrs. Edward (Margaret Ann Watson) Ford,Ford,0,5,0.0,2,test
161,Mr. William Neal Ford,Ford,0,5,0.0,2,test


,Name,Family,Survived,FamilySize,FamilySurvivalRate,ConfidenceFamilySurvival,Source
51,Mr. Mansour Hanna,Hanna,0,1,0.5,0,test


,Name,Family,Survived,FamilySize,FamilySurvivalRate,ConfidenceFamilySurvival,Source
279,Mr. Karl Ivar Sven Berglund,Berglund,0,1,0.5,0,train


Colonnes présentes dans df: Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings/Spouses Aboard',
       'Parents/Children Aboard', 'Fare', 'FamilySize', 'FamilyType',
       'NameLength', 'AgeGroup', 'Title_Master', 'Title_Military',
       'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Professional',
       'Title_Royalty', 'TicketFrequency', 'Family', 'FamilySurvivalRate',
       'ConfidenceFamilySurvival', 'Source'],
      dtype='object')


In [56]:
df_test['ConfidenceFamilySurvival'].value_counts()



ConfidenceFamilySurvival
0    102
1     37
2     18
3     14
4      4
7      2
5      1
Name: count, dtype: int64

In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
# Sélection des features
features = [
    'Pclass', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard',
    'Fare', 'FamilySize', 'FamilyType', 'NameLength', 'AgeGroup',
    'Title_Master', 'Title_Military', 'Title_Miss', 'Title_Mr', 'Title_Mrs',
    'Title_Professional', 'Title_Royalty', 'TicketFrequency',
 'FamilySurvivalRate', 'ConfidenceFamilySurvival',
]
X_train = df_train[features]
X_test = df_test[features]


# La cible/target
Y_train = df_train['Survived']
y_test = df_test['Survived']




In [58]:
from sklearn.preprocessing import StandardScaler
scaler_cols = ['Parents/Children Aboard','Siblings/Spouses Aboard', 'Fare', 'NameLength', 'FamilySize', 'NameLength',
               'TicketFrequency',  'FamilySurvivalRate', 'ConfidenceFamilySurvival',]

scaler = StandardScaler()
X_train[scaler_cols] = scaler.fit_transform(X_train[scaler_cols])
X_test[scaler_cols] = scaler.transform(X_test[scaler_cols])


/var/folders/90/vkxt4j7s77jgmpj5lqchw0bh0000gn/T/ipykernel_16116/2699645938.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[scaler_cols] = scaler.fit_transform(X_train[scaler_cols])
/var/folders/90/vkxt4j7s77jgmpj5lqchw0bh0000gn/T/ipykernel_16116/2699645938.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[scaler_cols] = scaler.transform(X_test[scaler_cols])


In [59]:
for col in X_train.columns:
    X_train[col] = X_train[col].astype(float)
    X_test[col] = X_test[col].astype(float)

/var/folders/90/vkxt4j7s77jgmpj5lqchw0bh0000gn/T/ipykernel_16116/3556882628.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = X_train[col].astype(float)
/var/folders/90/vkxt4j7s77jgmpj5lqchw0bh0000gn/T/ipykernel_16116/3556882628.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col] = X_test[col].astype(float)
/var/folders/90/vkxt4j7s77jgmpj5lqchw0bh0000gn/T/ipykernel_16116/3556882628.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


SQD testé initialement et très mauvais résultat donc ne sera pas testé par la suite

In [60]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
import random
import numpy as np
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
# Configuration des paramètres
input_size = X_train.shape[1]
output_size = 1
num_epochs = 100

# Définition du réseau de neurones avec plusieurs architectures
class TitanicNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, activation_fn, dropout_rate=0.5):
        super(TitanicNN, self).__init__()
        layers = []
        in_size = input_size
        for h_size in hidden_sizes:
            layers.append(nn.Linear(in_size, h_size))
            layers.append(activation_fn())
            layers.append(nn.Dropout(dropout_rate))  # Ajout de Dropout
            in_size = h_size
        layers.append(nn.Linear(in_size, output_size))
        layers.append(nn.Sigmoid())
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# Implémentation de Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        else:
            BCE_loss = nn.functional.binary_cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

# Plages de valeurs pour les hyperparamètres
param_ranges = {
    'hidden_sizes': [(16,), (32,), (16, 8), (32, 16), (64, 32, 16)],  # Différentes architectures
    'activation_fn': [nn.ReLU, nn.Tanh, nn.Sigmoid],  # Différentes fonctions d'activation
    'learning_rate': (0.001, 0.05),  # Plage réduite pour learning_rate
    'batch_size': (16, 64)  # Plage pour batch_size
}

loss_functions = {
    'BCELoss': nn.BCELoss(),
    'BCEWithLogitsLoss': nn.BCEWithLogitsLoss(),
    'FocalLoss': FocalLoss(logits=True)
}

# Détection de l'appareil (GPU si disponible)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fonction pour entraîner et valider le modèle
def train_and_validate_model(params, loss_fn, X_train, Y_train, optimizer_type, run_name):
    X_train_fold, X_val_fold, Y_train_fold, Y_val_fold = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

    X_train_tensor = torch.tensor(X_train_fold.values, dtype=torch.float32).to(device)
    Y_train_tensor = torch.tensor(Y_train_fold.values, dtype=torch.float32).unsqueeze(1).to(device)
    X_val_tensor = torch.tensor(X_val_fold.values, dtype=torch.float32).to(device)
    Y_val_tensor = torch.tensor(Y_val_fold.values, dtype=torch.float32).unsqueeze(1).to(device)

    train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=params['batch_size'], shuffle=True)

    model = TitanicNN(input_size, params['hidden_sizes'], output_size, params['activation_fn']).to(device)
    criterion = loss_fn

    if optimizer_type == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=params['learning_rate'])
    elif optimizer_type == 'AdaGrad':
        optimizer = optim.Adagrad(model.parameters(), lr=params['learning_rate'])
    elif optimizer_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    writer = SummaryWriter(f"runs/{run_name}")

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            writer.add_scalar(f'Loss/train', loss.item(), epoch * len(train_loader) + i)

        writer.add_scalar(f'Loss/train_epoch', running_loss / len(train_loader), epoch)

    model.eval()
    val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=params['batch_size'], shuffle=False)
    all_predictions = []
    all_labels = []
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            predictions = (outputs > 0.5).float()
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_accuracy = np.mean(np.array(all_predictions) == np.array(all_labels))
    mean_f1_score = f1_score(all_labels, all_predictions, zero_division=1)
    mean_precision = precision_score(all_labels, all_predictions, zero_division=1)
    mean_recall = recall_score(all_labels, all_predictions, zero_division=1)

    writer.add_scalar(f'Loss/val', val_loss / len(val_loader), epoch)
    writer.add_scalar(f'Accuracy/val', val_accuracy, epoch)
    writer.add_scalar(f'F1_Score/val', mean_f1_score, epoch)
    writer.add_scalar(f'Precision/val', mean_precision, epoch)
    writer.add_scalar(f'Recall/val', mean_recall, epoch)

    writer.close()

    return val_accuracy, mean_f1_score, mean_precision, mean_recall

# Recherche d'hyperparamètres avec random search
optimizers = ['AdaGrad', 'Adam']
num_iterations = 120  # Nombre d'itérations pour la recherche aléatoire

best_metrics = {optimizer: {
    'accuracy': 0.0,
    'f1_score': 0.0,
    'precision': 0.0,
    'recall': 0.0
} for optimizer in optimizers}
best_params = {optimizer: None for optimizer in optimizers}
best_loss_fn = {optimizer: None for optimizer in optimizers}

for optimizer in optimizers:
    run_name = "TitanicNN"  # Nom de l'exécution, peut être modifié selon les besoins
    for iteration in range(num_iterations):
        params = {
            'hidden_sizes': random.choice(param_ranges['hidden_sizes']),
            'activation_fn': random.choice(param_ranges['activation_fn']),
            'learning_rate': random.uniform(*param_ranges['learning_rate']),
            'batch_size': random.randint(*param_ranges['batch_size'])
        }

        loss_fn_name, loss_fn = random.choice(list(loss_functions.items()))

        hyperparams_str = f"{params['hidden_sizes']}_{params['activation_fn'].__name__}_{params['learning_rate']}_{params['batch_size']}_{loss_fn_name}"
        run_name = f"{optimizer}_{hyperparams_str}"

        val_accuracy, mean_f1_score, mean_precision, mean_recall = train_and_validate_model(params, loss_fn, X_train, Y_train, optimizer, run_name)

        writer = SummaryWriter(f"runs/{run_name}")

        writer.add_scalar(f'Accuracy/val', val_accuracy, iteration + 1)
        writer.add_scalar(f'F1_Score/val', mean_f1_score, iteration + 1)
        writer.add_scalar(f'Precision/val', mean_precision, iteration + 1)
        writer.add_scalar(f'Recall/val', mean_recall, iteration + 1)

        writer.close()

        if val_accuracy > best_metrics[optimizer]['accuracy']:
            best_metrics[optimizer].update({
                'accuracy': val_accuracy,
                'f1_score': mean_f1_score,
                'precision': mean_precision,
                'recall': mean_recall
            })
            best_params[optimizer] = params
            best_loss_fn[optimizer] = loss_fn_name

        print(f"Optimizer: {optimizer} | Iteration {iteration + 1}/{num_iterations}")
        print("Hyperparamètres :", params)
        print("Fonction de coût :", loss_fn_name)
        print("Accuracy :", val_accuracy)
        print("F1 Score moyen :", mean_f1_score)
        print("Précision moyenne :", mean_precision)
        print("Rappel moyen :", mean_recall)
        print()

for optimizer in optimizers:
    print(f"Meilleurs hyperparamètres pour {optimizer} :", best_params[optimizer])
    print(f"Meilleure fonction de coût pour {optimizer} :", best_loss_fn[optimizer])
    print(f"Meilleure accuracy pour {optimizer} :", best_metrics[optimizer]['accuracy'])
    print(f"Meilleur F1 Score moyen pour {optimizer} :", best_metrics[optimizer]['f1_score'])
    print(f"Meilleure précision moyenne pour {optimizer} :", best_metrics[optimizer]['precision'])
    print(f"Meilleur rappel moyen pour {optimizer} :", best_metrics[optimizer]['recall'])
    print()


Optimizer: AdaGrad | Iteration 1/120
Hyperparamètres : {'hidden_sizes': (16,), 'activation_fn': <class 'torch.nn.modules.activation.ReLU'>, 'learning_rate': 0.03733597448823182, 'batch_size': 31}
Fonction de coût : BCELoss
Accuracy : 0.823943661971831
F1 Score moyen : 0.7663551401869159
Précision moyenne : 0.8367346938775511
Rappel moyen : 0.7068965517241379

Optimizer: AdaGrad | Iteration 2/120
Hyperparamètres : {'hidden_sizes': (32,), 'activation_fn': <class 'torch.nn.modules.activation.Sigmoid'>, 'learning_rate': 0.006022263632403868, 'batch_size': 63}
Fonction de coût : FocalLoss
Accuracy : 0.5915492957746479
F1 Score moyen : 0.0
Précision moyenne : 1.0
Rappel moyen : 0.0

Optimizer: AdaGrad | Iteration 3/120
Hyperparamètres : {'hidden_sizes': (16,), 'activation_fn': <class 'torch.nn.modules.activation.Sigmoid'>, 'learning_rate': 0.021674169164578254, 'batch_size': 17}
Fonction de coût : BCELoss
Accuracy : 0.8098591549295775
F1 Score moyen : 0.7428571428571429
Précision moyenne : 0

In [61]:
import torch
import numpy as np
import random

# Fixer les graines pour la reproductibilité
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

def evaluate_best_model(best_params, best_loss_fn, X_train, Y_train, X_test, Y_test, optimizer_type):
    set_seed()

    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
    Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.float32).unsqueeze(1).to(device)
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
    Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.float32).unsqueeze(1).to(device)

    train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=best_params[optimizer_type]['batch_size'], shuffle=True)

    model = TitanicNN(input_size, best_params[optimizer_type]['hidden_sizes'], output_size, best_params[optimizer_type]['activation_fn']).to(device)
    criterion = loss_functions[best_loss_fn[optimizer_type]]

    if optimizer_type == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=best_params[optimizer_type]['learning_rate'])
    elif optimizer_type == 'AdaGrad':
        optimizer = optim.Adagrad(model.parameters(), lr=best_params[optimizer_type]['learning_rate'])
    elif optimizer_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=best_params[optimizer_type]['learning_rate'])

    run_name = f"Best_Model_{optimizer_type}"

    writer = SummaryWriter(f"runs/{run_name}")

    writer.add_graph(model, X_train_tensor[:1]) 

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        writer.add_scalar('Loss/train_epoch', running_loss / len(train_loader), epoch)

    # Évaluer le modèle sur le jeu de test
    model.eval()
    test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=best_params[optimizer_type]['batch_size'], shuffle=False)
    all_predictions = []
    all_labels = []
    test_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            predictions = (outputs > 0.5).float()
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    test_accuracy = np.mean(np.array(all_predictions) == np.array(all_labels))
    test_f1_score = f1_score(all_labels, all_predictions, zero_division=1)
    test_precision = precision_score(all_labels, all_predictions, zero_division=1)
    test_recall = recall_score(all_labels, all_predictions, zero_division=1)

    writer.add_scalar('Loss/test', test_loss / len(test_loader), epoch)
    writer.add_scalar('Accuracy/test', test_accuracy, epoch)
    writer.add_scalar('F1_Score/test', test_f1_score, epoch)
    writer.add_scalar('Precision/test', test_precision, epoch)
    writer.add_scalar('Recall/test', test_recall, epoch)

    writer.close()

    print(f"Évaluation sur le jeu de test pour {optimizer_type} :")
    print(f"Accuracy : {test_accuracy}")
    print(f"F1 Score : {test_f1_score}")
    print(f"Précision : {test_precision}")
    print(f"Rappel : {test_recall}")

# Évaluer les meilleurs modèles pour chaque optimiseur
for optimizer in optimizers:
    print(best_params[optimizer])
    evaluate_best_model(best_params, best_loss_fn, X_train, Y_train, X_test, y_test, optimizer)


{'hidden_sizes': (64, 32, 16), 'activation_fn': <class 'torch.nn.modules.activation.ReLU'>, 'learning_rate': 0.019874576233891633, 'batch_size': 52}
Évaluation sur le jeu de test pour AdaGrad :
Accuracy : 0.8202247191011236
F1 Score : 0.7241379310344828
Précision : 0.7636363636363637
Rappel : 0.6885245901639344
{'hidden_sizes': (32, 16), 'activation_fn': <class 'torch.nn.modules.activation.Tanh'>, 'learning_rate': 0.010761632956335726, 'batch_size': 40}
Évaluation sur le jeu de test pour Adam :
Accuracy : 0.8089887640449438
F1 Score : 0.6964285714285714
Précision : 0.7647058823529411
Rappel : 0.639344262295082
